## Image Gradient:

An image gradient is a directional change in the intensity or color(pixel value in the image matrix) in an image. The gradient of the image is one of the fundamental building blocks in image processing.

Mathematically, the gradient(let G) of a two-variable function (here the image intensity function) at each image point is a 2D vector with the components given by the derivatives in the horizontal( let Gx) and vertical directions (let Gy). At each image point, the gradient vector points in the direction of largest possible intensity increase, and the length of the gradient vector corresponds to the rate of change in that direction.

![title](44.png)


Since **the intensity function of a digital image is only known at discrete points, derivatives of this function cannot be defined** unless we assume that there is an underlying continuous intensity function.
Hence all the operators actually give the approximate value of the image in X and Y direction for all channels separately.


## Sobel Derivative

One of the most important convolutions is the computation of derivatives in an image (or an approximation to them since image function is not a continous but a dicrete function).

Why do we need derivative of an image? Let's say we need to find edges in a picture and we know that there is a sharp(usually sharp) intensity change in the direction perpendicular to the edge.

In the follwing image, we can see intensity change from black to white, perpendicular to the edge:
![title](40.png)


You can easily notice that in an edge, the pixel intensity changes in a notorious way. A good way to express changes is by using derivatives. A high change in gradient indicates a major change in the image.

So edge detection can be performed by :
We can deduce that a method to detect edges in an image can be performed by locating pixel locations where the gradient is higher than its neighbors (or to generalize, higher than a threshold).


### SOBEL OPERATOR:
The Sobel Operator is a discrete differentiation operator. It computes an approximation of the gradient of an image intensity function.
The Sobel Operator combines Gaussian smoothing and differentiation.

FORMULA:

Assuming that the image to be operated is I:

1.We calculate two derivatives: <br/>
&nbsp;&nbsp;&nbsp;&nbsp;a.Horizontal changes: This is computed by convolving I with a kernel G_{x} with odd size. For example for a kernel size of 3, G_{x} would be computed as:
  
  ![title](41.png)
  
&nbsp;&nbsp;&nbsp;&nbsp; b.Vertical changes: This is computed by convolving I with a kernel G_{y} with odd size. For example for a kernel size of 3, G_{y} would be computed as:

 ![title](42.png)
 
2.At each point of the image we calculate an approximation of the gradient in that point by combining both results above:

 ![title](43.png)
 
 
**NOTE: '\*' is used here to represent convolution not for multiplication.**


## Laplacian Operator

1. Sobel Operator was based on the fact that in the edge area, the pixel intensity shows a “jump” or a high variation of intensity. Getting the first derivative of the intensity, we observed that an edge is characterized by a maximum, as it can be seen in the figure:


 ![title](52.jpg)
 
2. And...what happens if we take the second derivative?

![title](53.jpg)


You can observe that the second derivative is zero! So, we can also use this criterion to attempt to detect edges in an image. However, note that zeros will not only appear in edges (they can actually appear in other meaningless locations); this can be solved by applying filtering where needed.

Formulation:

1. From the explanation above, we deduce that the second derivative can be used to detect edges. Since images are “2D”, we would need to take the derivative in both dimensions. Here, the Laplacian operator comes handy.

2. The Laplacian operator is defined by:

![title](54.png)

3. The Laplacian operator is implemented in OpenCV by the function Laplacian. In fact, since the Laplacian uses the gradient of images, it calls internally the Sobel operator to perform its computation.







In [ ]:
import cv2
import numpy as np 


In [ ]:
img = cv2.imread("46.png" , cv2.IMREAD_COLOR)

"""
After loading the image the process goes in three steps:
1. Convert the image to GRAY_SCALE
2. Remove the noise from tha image using gaussion filter or laplacian(better results)
3. Apply Sobel Operator on the result
"""

gray_image = cv2.cvtColor(img , cv2.COLOR_BGR2GRAY)

laplacian = cv2.Laplacian(gray_image , cv2.CV_64F)

#blur = cv2.GaussianBlur(gray_image , (15,15) , 0)

sobelx = cv2.Sobel(laplacian,cv2.CV_64F,1,0,ksize=5)

sobely = cv2.Sobel(laplacian , cv2.CV_64F , 1, 0 , ksize = 5)

In [ ]:
while True:
    cv2.imshow("image" , img)
    cv2.imshow("laplacian" , laplacian)
    cv2.imshow("sobelx" , sobelx)
    cv2.imshow("sobely" , sobely)
    
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
        
cv2.destroyAllWindows()
for i in range(5):
    cv2.waitKey(0)
    

RESULT:

Image:

![title](46.png)

Laplacian:

![title](47.png)

SobelX:

![title](48.png)

SobelY:

![title](49.png)



We arent able to see much difference between sobelx and sobely output. Here is an example from Google images:

Sobel X : Applying Sobel Operator in the X direction

![title](51.jpg)


Sobel Y : Applying Sobel Operator in the Y direction

![title](50.jpg)

## Canny Edge Detection

Applying a Sobel Operator on an image produces an image with all egdes in white and the background in black. But sobel detect/ show all the minute edges present in the image which may or may not be relevant to us. The extraction of the relevant edges and thinning them is done by Canny Edge Detection.

The output of the Sobel Operator is fed into the Canny Egde Detector which produces an image with relevant edges which can be used to detect objects inside the image.

**Canny Edge Detection** is a popular edge detection algorithm. It was developed by John F. Canny in 1986. It is a multi-stage algorithm and we will go through each stages.
OpenCV functions for that : **cv2.Canny()**

1. **Noise Reduction:**
Since edge detection is susceptible to noise in the image, first step is to remove the noise in the image with a 5x5 Gaussian filter. We have already seen this in previous chapters.

2. **Finding Intensity Gradient of the Image:**
Smoothened image is then filtered with a Sobel kernel in both horizontal and vertical direction to get first derivative in horizontal direction (G_x) and vertical direction (G_y). From these two images, we can find edge gradient and direction for each pixel as follows:

![title](61.png)

Gradient direction is always perpendicular to edges. It is rounded to one of four angles representing vertical, horizontal and two diagonal directions.

3. **Non-maximum Suppression :**
After getting gradient magnitude and direction, a full scan of image is done to remove any unwanted pixels which may not constitute the edge. For this, at every pixel, pixel is checked if it is a local maximum in its neighborhood in the direction of gradient. Check the image below:

![title](62.jpg)

Point A is on the edge ( in vertical direction). Gradient direction is normal to the edge. Point B and C are in gradient directions. So point A is checked with point B and C to see if it forms a local maximum. If so, it is considered for next stage, otherwise, it is suppressed ( put to zero).

In short, the result you get is a binary image with “thin edges”.

3. **Hysteresis Thresholding :**
This stage decides which are all edges are really edges and which are not. For this, we need two threshold values, minVal and maxVal. Any edges with intensity gradient more than maxVal are sure to be edges and those below minVal are sure to be non-edges, so discarded. Those who lie between these two thresholds are classified edges or non-edges based on their connectivity. If they are connected to “sure-edge” pixels, they are considered to be part of edges. Otherwise, they are also discarded. See the image below:

![title](63.jpg)

The edge A is above the maxVal, so considered as “sure-edge”. Although edge C is below maxVal, it is connected to edge A, so that also considered as valid edge and we get that full curve. But edge B, although it is above minVal and is in same region as that of edge C, it is not connected to any “sure-edge”, so that is discarded. So it is very important that we have to select minVal and maxVal accordingly to get the correct result.

This stage also removes small pixels noises on the assumption that edges are long lines.

So what we finally get is strong edges in the image.

OpenCV puts all the above in single function, cv2.Canny(). We will see how to use it. First argument is our input image. Second and third arguments are our minVal and maxVal respectively. Third argument is aperture_size. It is the size of Sobel kernel used for find image gradients. By default it is 3. Last argument is L2gradient which specifies the equation for finding gradient magnitude. If it is True, it uses the equation mentioned above which is more accurate, otherwise it uses this function: Edge\_Gradient : (G) = |G_x| + |G_y|. By default, it is False.
